Во многих популярных курсах машинного и глубокого обучения вас научат классифицировать собак и кошек, предсказывать цены на недвижимость, покажут еще десятки задач, в которых машинное обучение, вроде как, отлично работает. Но вам расскажут намного меньше (или вообще ничего) о тех случаях, когда ML-модели не работают так, как ожидалось.

Частой проблемой в машинном обучении является неспособность ML-моделей корректно работать на большем разнообразии примеров, чем те, что встречались при обучении. Здесь идет речь не просто о других примерах (например, тестовых), а о других *типах* примеров. Например, сеть обучалась на изображениях коровы, в которых чаще всего корова был на фоне травы, а при тестировании требуется корректное распознавание коровы на любом фоне. Почему ML-модели часто не справляются с такой задачей и что с этим делать - мы рассмотрим далее. Работа над этой проблемой важна не только для решения практических задач, но и в целом для дальнейшего развития ИИ.

Конечно, этим проблемы машинного обучения не ограничиваются, существуют также сложности с интерпретацией моделей, проблемы предвзятости и этики, ресурсоемкости обучения и прочие. В данном обзоре рассмотрим только проблемы обобщения.

## Содержание

- Проблемы обобщения
    - Задача обобщения в машинном обучении
    - Утечка данных
    - Shortcut learning: "right for the wrong reasons"
    - Shortcut learning в языковых моделях
    - Уровни обобщения моделей машинного обобщения
    - Состязательные атаки
    - Проблема неконкретизации в ML-задачах
    - Выводы
- Способы решения проблем обобщения
    - Стресс-тесты для диагностики работы модели
    - Доменная адаптация
    - Еще больше данных?
    - Более крупные или более эффективные модели?
    - Модификации способов обучения языковых моделей
    - Архитектура моделей и структура данных
    - Направления исследований
    - Общий искусственный интеллект
- Заключение

## Проблемы обобщения

### Задача обобщения в машинном обучении

Задача машинного обучения заключается в написании алгоритмов, которые автоматически выводят общие закономерности из частных случаев (обучающих примеров). Этот процесс называется обобщением (*generalization*), или индукцией. Часто требуется найти зависимость между исходными данными $X$ и целевыми данными $y$ в виде функции $f: X \to y$.

Например, требуется определить по фотографии тип или положение изображенного объекта. Для решения этой задачи можно собрать большое количество размеченных данных (*supervised learning*), можно также использовать ненадежно размеченные данные (*weakly-supervised learning*), например фотографии из Instagram c хештегами, или даже неразмеченные данные (*self-supervised learning*). Но в любом случае мы используем некий набор конкретных примеров.

Понятно, что чем больше доступно примеров, тем выше будет качество полученного решения  (при прочих равных условиях). Больше данных - выше точность. Все логично?

Существуют даже теоремы, доказывающие для различных ML-алгоритмов стремление получаемого решения к идеалу при неограниченном увеличении количества обучающих данных и размера модели (это свойство называется *universal consistency*; например, для нейронных сетей см. [Faragó and Lugosi, 1993]($Strong Universal Consistency of Neural Network Classifiers$)). При доказательстве подобных теорем считается, что данные $X, y$ берутся из некоего фиксированного, но неизвестного распределения $P(X, y)$. Такой подход называется *statistical learning framework*, он излагается почти в любой книге по машинному обучению (например, см. [Deep Learning book]($Deep Learning$), раздел [5.2](https://www.deeplearningbook.org/contents/ml.html), есть также русское издание).

С развитием технологий стали доступны дешевые вычисления и огромные объемы данных, что позволило эффективно решать очень сложные задачи. Например, языковая модель [GPT-3](https://habr.com/ru/post/514698/) ([Brown et al., 2020]($Language Models are Few-Shot Learners$)) от компании OpenAI обучалась на 570 Гб текстов и имеет 175 миллиардов параметров, по [приблизительным подсчетам](https://www.reddit.com/r/MachineLearning/comments/hwfjej/d_the_cost_of_training_gpt3/) обучение стоило миллионы долларов. Примеры текстов, которые генерирует GPT-3, можно найти [здесь](https://twitter.com/raphaelmilliere/status/1289129723310886912), [здесь](https://twitter.com/raphaelmilliere/status/1289135398644580352), [здесь](https://www.gwern.net/GPT-3) и [здесь](https://tech.liga.net/technology/article/ne-v-moih-interesah-vas-unichtojit-neyroset-gpt-3-naehala-na-tsukerberga-eto-konets). В Google обучили сеть с 1.6 триллиона параметров ([Fedus et al., 2021]($Switch Transformers: Scaling to Trillion Parameter Models with Simple and Efficient Sparsity$)), [в Китае](https://habr.com/ru/news/t/560854/) - сеть с 1.75 триллиона параметров, и это далеко не предел: уже [делаются попытки](https://www.zdnet.com/article/cerebras-prepares-for-the-era-of-120-trillion-parameter-neural-networks/) обучать сеть со 120 триллионами параметров, что примерно равно количеству синапсов в [человеческом мозге](https://insh.world/science/elaborate-visualization-human-brain-date/). Более крупные модели и больше обучающих данных - выше точность.

Казалось бы, что может пойти не так?

Рассмотрим несколько примеров, когда простого увеличения объема обучающих данных может оказаться недостаточно. Начнем с проблемы утечки данных, и затем рассмотрим более общую проблему ограниченного разнообразия обучающих данных и ситуации, когда модель лишь "делает вид", что обучается.

### Утечка данных

Утечкой данных называется ситуация, когда существует некий признак, который при обучении содержал больше информации о целевой переменной, чем при последующем применении модели на практике. Утечка данных может возникать в самых разных формах.

**Пример 1.** При диагностике заболеваний по рентгеновским снимкам модель обучается и тестируется на данных, собранных с разных больниц. Рентгеновские аппараты в разных больницах выдают снимки с немного разной тональностью, при этом различия могут быть даже незаметны человеческим глазом. Модель может обучиться определять больницу на основе тональности снимка, а значит и вероятный диагноз (в разных больницах лежат люди с разными диагнозами), при этом вообще не анализируя изображение на снимке. Такая модель покажет отличную точность при тестировании, так как тестовые снимки взяты из той же выборки данных.

**Пример 2.** В задаче диагностики [реальных и фейковых вакансий](https://www.kaggle.com/shivamb/real-or-fake-fake-jobposting-prediction) почти все фейковые вакансии относились к Европе. Обученная на этих данных модель может не считать подозрительными любые вакансии из других регионов. При применении на практике такая модель станет бесполезна, как только фейковые вакансии станут размещаться в других регионах. Конечно можно удалить регион из признаков, но тогда модель будет пытаться предсказать регион по другим признакам (скажем, по отдельным словам в описании и требованиях), и проблема сохранится.

**Пример 3.** В той же задаче есть признак "имя компании", и у компаний может быть по нескольку вакансий. Если компания фейковая, то все ее вакансии фейковые. Если мы разделим данные на обучающую и тестовую часть (или на фолды) таким образом, что вакансии одной и той же компании попадут и в обучающую, и в тестовую часть, то модели достаточно будет запомнить имена фейковых компаний для предсказания на тестовой выборке, в результате на тесте мы получим сильно завышенную точность.

<img src="assets/leakage.jpg" width="300" align="center">

Все описанные проблемы нельзя решить просто увеличением количества обучающих данных без изменения подхода к обучению. Здесь можно возразить, что утечка данных не является проблемой самих ML-алгоритмов, поскольку алгоритм и не может знать о том, какой признак нужно учитывать, а какой нет. Но давайте посмотрим на другие похожие примеры.

### Shortcut learning: "right for the wrong reasons"

В начале XX века жила лошадь по имени Умный Ганс, которая якобы умела решать сложные арифметические задачи, постукивая копытом нужное число раз. В итоге оказалось, что лошадь определяет момент, когда нужно прекращать стучать копытом, по выражению лица того, кто задает вопрос. С тех пор имя "умного Ганса" стало нарицательным ([1]($Unmasking Clever Hans Predictors and Assessing What Machines Really Learn$), [2]($When Choosing Plausible Alternatives, Clever Hans can be Clever$)) - оно означает получение решения обходным, нечестным путем, не решая при этом саму задачу в том смысле, какой мы хотим. Такое обходное "решение" внезапно перестает работать, если меняются условия (например, человек, задающий вопрос лошади, сам не знает ответа на него).

Исследователи давно заметили, что многие современные модели глубокого обучения похожи на "умного Ганса". Вспомним хотя бы легендарную [статью](https://habr.com/ru/post/259191/) на Хабре про "леопардовый диван":

> ... Где-то в этот момент в мозг начинает прокрадываться ужасная догадка — а что если вот это различие в текстуре пятен и есть главный критерий, по которому алгоритм отличает три возможных класса распознавания [леопарда, ягуара, гепарда] друг от друга? То есть на самом деле сверточная сеть не обращает внимания на форму изображенного объекта, количество лап, толщину челюсти, особенности позы и все вот эти тонкие различия, которые, как мы было предположили, она умеет понимать — и просто сравнивает картинки как два куска текстуры?

> Это предположение необходимо проверить. Давайте возьмем для проверки простую, бесхитростную картинку, без каких-либо шумов, искажений и прочих факторов, осложняющих жизнь распознаванию. Уверен, эту картинку с первого взгляда легко опознает любой человек. <br> <br> <img src='assets/leopard.jpg' width=250/>

> ... Похоже, что наша догадка насчет текстуры близка к истине. ... Я сделал еще пару проверок (Microsoft, Google) — некоторые из них ведут себя лучше, не подсовывая ягуара, но победить повернутый набок диван не смог никто. Неплохой результат в мире, где уже мелькают заголовки в духе "{Somebody}'s Deep Learning Project Outperforms Humans In Image Recognition".

В самом деле, зачем модели, решающей задачу классификации, выучивать форму и другие комплексные свойства объекта, называемого "леопард", если простое присутствие нужной текстуры на изображении дает почти 100%-ю точность выявления леопарда в обучающей (и тестовой) выборке из ImageNet?

То же самое касается предсказания объекта с учетом фона. Мы не закладывали при обучении никакой информации о том, на основании чего мы хотим получать предсказание: на основании объекта или окружающего его фона. Было даже обнаружено ([Zhu et al., 2016]($Object Recognition with and without Objects$)), что нейронные сети способны с хорошей точностью предсказывать класс объекта, если сам объект вырезан и оставлен только фон.

**Shortcut learning** - явление, когда модели получают верный ответ с помощью неверных в общем случае рассуждений ("right for the wrong reasons"), которые хорошо работают только для обучающего распределения данных. Поскольку обучающая и тестовая выборка обычно берутся из одного распределения, то такие модели могут давать хороую точность и при тестировании.

> Shortcuts are decision rules that perform well on standard benchmarks but fail to transfer to more challenging testing conditions, such as real-world scenarios.

Другие примеры можно найти в замечательной статье Shortcut Learning in Deep Neural Networks ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)), опубликованной в журнале Nature и основанной на обобщении информации из более чем 140 источников.

Смена угла поворота объекта, появление его в необычном (с точки зрения обучающей выборки) окружении, добавление необычного шума может полностью испортить работу модели. На иллюстрации ниже слева показаны пары изображений, одинаково классифицируемые с точки зрения человека, но совершенно разные с точки зрения сверточной нейронной сети. Справа показаны, наоборот, изображения, имеющие одинаковый класс для нейронной сети и разный для человека.

<img src="assets/generalisation.jpg" width="800" align="center">

В целом исследователями был сделан вывод о том, что многие современные сверточные нейронные сети предпочитают ориентироваться на локальные участки текстур ([Jo and Bengio, 2017]($Measuring the tendency of CNNs to Learn Surface Statistical Regularities$); [Geirhos et al., 2018]($ImageNet-trained CNNs are biased towards texture; increasing shape bias improves accuracy and robustness$)), которые могут относиться как к объекту, так и к фону, и не классифицируют объект на основании его общей формы ([Baker et al., 2018]($Deep convolutional networks do not classify based on global object shape$)).

[Jo and Bengio, 2017]($Measuring the tendency of CNNs to Learn Surface Statistical Regularities$) используют фильтрацию, основанную на преобразовании Фурье, для внесения в изображения помех, которые не мешают распознаванию их человеком, но сильно снижают качество работы сверточных нейронных сетей. Увеличение глубины сетей не избавляет от этой проблемы. Более того, если добавить в обучающий датасет один из типов Фурье-искажений, то это не делает сеть устойчивее к другим типам Фурье-искажений (которых может быть очень много).

> The current incarnation of deep neural networks exhibit a tendency to learn surface statistical regularities as opposed to higher level abstractions in the dataset. For tasks such as object recognition, ... [this is] sufficient for high performance generalization, but in a narrow distributional sense.

>  To this end, we draw an analogy to adversarial training: augmenting the training set with a specific subset of adversarial examples does not make the network immune to adversarial examples in general. ...we do not believe that this sort of data augmentation is sufficient for
learning higher level abstractions in the dataset.

### Shortcut learning в языковых моделях

Та же проблема есть и в языковых моделях. Например, модель BERT ([Devlin et al., 2018]($BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding$)) обучена на таком объеме текстов, который человек не прочитает за всю жизнь, но при этом она в основном выучивает поверхностные, стереотипные ассоциации слов друг с другом (word co-occurence), и вряд ли хорошо понимает смысл текста.

> Some of BERT's world knowledge success comes from learning stereotypical associations, e. g. a person with an Italian-sounding name is predicted to be Italian, even when it is incorrect. ([Rogers et al., 2020]($A Primer in BERTology: What we know about how BERT works$))

Попробуем протестировать некоторые из наиболее популярных на сегодняшний день языковых моделей: RoBERTa ([Liu et al., 2019]($RoBERTa: A Robustly Optimized BERT Pretraining Approach$)), ALBERT ([Lan et al., 2019]($ALBERT: A Lite BERT for Self-supervised Learning of Language Representations$)) и mT5 ([Xue et al., 2020]($mT5: A massively multilingual pre-trained text-to-text transformer$)), взяв для тестирования их крупные варианты из репозитория HuggingFace: [roberta-large](https://huggingface.co/roberta-large), [albert-xxlarge-v2](https://huggingface.co/albert-xxlarge-v2) и [mt5-xl](https://huggingface.co/google/mt5-xl). Эти модели обучались на разных задачах, в числе которых была задача предсказания закрытых маской слов в тексте, называемая masked language model objective, или cloze task. Проверим их работу на различных примерах.

*Примечание.* Вы можете сами повторить эксперимент, используя окно "Hosted inference API" по приведенным выше ссылкам; для mT5 можно использовать [этот ноутбук](https://drive.google.com/file/d/1XeGrS4MAroR9pFSytwbB9wMx2KVx9bzC/view?usp=sharing), требуется GPU с большим объемом памяти.

<img src="assets/mlm.jpg" width="600" align="center">

Как видим, модели иногда неплохо запоминают простые факты, но при этом все рассмотренные модели часто делают ошибки там, где нужно понимать логику повествования. Если логика повествования противоречит статистике совместной встречаемости слов, то модели обычно предпочитают статистику. Например, в последних двух примерах модель ALBERT вероятно работает так: если "approaches", то "discuss", если "approach", то "propose" - потому при обучении часто встречались такие сочетания. Слово "existing" при этом ее не смущает.

Здесь можно возразить, что эти модели не обучались генерировать тексты без логических ошибок, они обучались воспроизводить реальные тексты, даже если в них есть ошибки. Но даже с учетом этого, те варианты слов, которые выбирают модели, часто оказываются абсурдны и практически не могли встретиться в написанных человеком текстах.

Вполне вероятно, что будь модели еще в 100 раз крупнее, обучаясь в 100 раз дольше и на большем объеме данных - они не совершили бы таких очевидных ошибок. Но даже эти модели очень большие (3.7 миллиарда параметров в mT5-XL) и обучались на сотнях гигабайт текстов, чему качество их работы на нетипичных примерах явно не соответствует - это наводит на мысль о неоптимальности подхода в целом.

Проблема shortcut learning присутствует и в тех моделях, которые специально дообучались для [лидербордов](https://gluebenchmark.com/leaderboard) GLUE ([Wang et al., 2018]($GLUE: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding$)) и SuperGLUE ([Wang et al., 2019]($SuperGLUE: A Stickier Benchmark for General-Purpose Language Understanding Systems$)). Эти бенчмарки нацелены на измерение уровня "понимания естественного языка" моделями машинного обучения. SuperGLUE содержит 8 задач, на некоторых из которых уже побит уровень человека (имеется в виду средний уровень человека, отличающегося от разметчика, т. к. разметка тоже делалась людьми).

Но действительно ли уровень человека побит? Как выясняется, высокая метрика качества в SuperGLUE достигается в основном с помощью shortcut learning, при этом модели эксплуатируют систематические проблемы в разметке датасета для предсказания ответа "обходным путем" ([Gururangan et al., 2018]($Annotation Artifacts in Natural Language Inference Data$); [Du et al., 2021]($Towards Interpreting and Mitigating Shortcut Learning Behavior of NLU Models$)), и мнение многих исследователей ([1](https://youtu.be/MlnU-fsWcVU?t=1150), [2](https://www.facebook.com/yann.lecun/posts/10157253205637143), [3](https://twitter.com/garymarcus/status/1344694003074179074), [4](https://venturebeat.com/2020/06/01/ai-machine-learning-openai-gpt-3-size-isnt-everything/) и др.) сходится в том, что на текущий момент языковые модели-трансформеры далеки от понимания смысла текстов в том смысле, в каком их понимает человек. Во введении мы рассматривали языковую модель GPT-3, имеющую огромный размер и генерирующую реалистичные тексты. Вот, что о ней [думает](https://www.facebook.com/yann.lecun/posts/10157253205637143) Ян Лекун:

> Some people have completely unrealistic expectations about what large-scale language models such as GPT-3 can do. ... GPT-3 doesn't have any knowledge of how the world actually works. *It only appears to have some level of background knowledge, to the extent that this knowledge is present in the statistics of text. But this knowledge is very shallow and disconnected from the underlying reality. ...trying to build intelligent machines by scaling up language models is like building a high-altitude airplanes to go to the moon*. You might beat altitude records, but going to the moon will require a completely different approach.

<img src="assets/gpt3world.jpg" width="450" align="center">

Как мне кажется, современные языковые модели вроде GPT-3 не стоит переоценивать, но и не стоит недооценивать. Со многими задачами они действительно очень хорошо справляются. Поскольку данный обзор посвящен проблемам машинного обучения, то в нем делается акцент на недостатках текущих ML-моделей (а значит и путях дальнейших исследований), но преимуществ у них тоже много.

### Уровни обобщения моделей машинного обобщения

Почти любой датасет имеет ограниченное разнообразие и не покрывает всех ситуаций, в которых желательна корректная работа модели. Особенно это проявляется в случае сложных данных, таких как изображения, тексты и звукозаписи. В данных могут существовать "паразитные корреляции" (*spurious correlations*), позволяющие с хорошей точностью предсказывать ответ только на данной выборке без "комплексного" понимания изображения или текста.

Это как раз и есть рассмотренные выше предсказания на основе фона, отдельных текстурных пятен или отдельных рядом стоящих слов. Датасеты, в которых паразитные корреляции явно выражены, называются *biased* (предвзятыми). Пожалуй, что все датасеты в задачах CV и NLP в той или иной степени предвзяты.

В статье о shortcut learning ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)) авторы рассматривают несколько уровней обобщения, которые могут достигать модели машинного обучения:

<img src="assets/taxonomy.jpg" width="340" align="center">

**Uninformative features**. Сеть использует признаки, которые не позволяют эффективно предсказывать ответ даже на обучающей выборке, например, случайно инициализированные веса нейронной сети.

**Overfitting features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на обучающей выборке, но не на всем распределении, из которого получена эта выборка. Под "распределением" здесь понимается вероятностное совместное распределение $P(x, y)$, из которого взяты данные (более подробно см. также [этот обзор](https://habr.com/ru/company/ods/blog/599573/), раздел "распределение данных").

**Shortcut features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ на распределении данных $P(x, y)$, из которого взята обучающая (и, как правило, тестовая) выборка. Поскольку выборка является набором независимых примеров, используется термин *independent and identically distributed ([i. i. d.](https://ru.wikipedia.org/wiki/%D0%9D%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D1%8B%D0%B5_%D0%BE%D0%B4%D0%B8%D0%BD%D0%B0%D0%BA%D0%BE%D0%B2%D0%BE_%D1%80%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D1%91%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D1%8B%D0%B5_%D0%B2%D0%B5%D0%BB%D0%B8%D1%87%D0%B8%D0%BD%D1%8B))*. Способность алгоритма с хорошей точностью работать на некоем фиксированном распределении данных авторы называют *i. i. d. обобщением*. Как мы видели выше, для этого вовсе не обязательно уметь решать саму поставленную задачу в общем виде, часто можно использовать "обходные пути" или утечки данных.

**Intended features**. Сеть использует признаки, которые позволяют эффективно предсказывать ответ в общем случае. Такие признаки будут хорошо работать и вне обучающего распределения данных, когда "обходные пути" закрыты (например, объект находится на необычном фоне, в необычной позиции, имеет необычную текстуру и пр.).

Ситуация, когда распределение данных отличается при обучении и применении называется сдвигом распределения данных (*distributional shift*). Иными словами, данные, на которых модель будет применяться, среднестатистически отличаются от тех, на которых модель обучалась и тестировалась. В частности, может идти речь о появлении новых типов примеров или изменении соотношения количества примеров разных типов. Это происходит из-за того, что обучающие данные часто недостаточно разнообразны и не покрывают все те типы примеров, на которых желательна корректная работа модели (либо покрывают их в неправильном соотношении).

Хорошая работа модели в условиях сдвига данных называется *out-of-distribution обобщением* (*OOD generalization*), она также известна под названием *domain generalization* ([Zhou et al., 2021]($Domain Generalization in Vision: A Survey$); [Wang et al., 2021]($Generalizing to Unseen Domains: A Survey on Domain Generalization$); см. также на [paperswithcode.com](https://paperswithcode.com/task/domain-generalization)), очевидна также ее связь с преодолением проблемы shortcut learning.

> If an image classifier was trained on photo images, would it work on sketch images? What if a car detector trained using urban images is tested in rural environments? Is it possible to deploy a semantic segmentation model trained using sunny images under rainy or snowy weather conditions? Can a health status classifier trained using one patient’s electrocardiogram data be used to diagnose another patient’s health status? ... Indeed, without access to target domain data, training a generalizable model that can work effectively in any unseen target domain is arguably one of the hardest problems in machine learning. ([Zhou et al., 2021]($Domain Generalization in Vision: A Survey$))

Еще один типичный пример - системы распознавания лиц. В обучающем датасете, даже очень большом, может ни быть ни одного изображения человеческого лица, на которое нанесен некий рисунок. Модель, обученная на таком датасете, научится быть инвариантной к положению лица в кадре и его выражению, и почти безошибочно идентифицировать человека. Однако модель может перестать корректно работать в случае, если надеты необычные [очки](https://www.theverge.com/2016/11/3/13507542/facial-recognition-glasses-trick-impersonate-fool) или на лицо нанесен [рисунок](https://www.rbth.com/science_and_tech/2017/07/22/hiding-from-artificial-intelligence-in-the-age-of-total-surveillance_808692), потому что *изображений такого типа не встречалось при обучении* (либо они встречались чрезвычайно редко и практически не оказали влияния на обучение).

В результате люди придумывают все новые и новые способы обманывать системы распознавания лиц, не пряча при этом полностью лицо. Модель не встречалась с такими изображениями при обучении и распознает лица неправильно, даже если человек на месте модели легко опознал бы знакомое лицо.

<img src="assets/facerec.jpg" width="550" align="center">

Конечно, распознавание лиц может не всегда использоваться во благо обществу, но здесь речь идет об общей проблеме систем машинного обучения. Эффективное out-of-distribution обобщение остается открытой проблемой, которую необходимо решать, если мы хотим дальнейшего развития ИИ. В целом, ее можно описать так:

> Generalizing beyond one's experiences – a hallmark of human intelligence from infancy – remains a formidable challenge for modern AI. ([Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$))

### Состязательные атаки

С работы [Szegedy et al., 2013]($Intriguing properties of neural networks$) начались исследования в области так называемых состязательных атак на нейронные сети (*adversarial attacks*). Авторы показали, что наложением на изображение незначительного шума можно заставить нейронные сети ошибаться в задаче классификации, предсказывая совсем другой класс. Конечно, для этого подходит не любой шум, а специальный паттерн шума, найти который можно с помощью оптимизации.

Еще более неожиданным оказалось то, что в точности тот же паттерн шума заставлял ошибаться на том же изображении и другие сети, обученные с другими гиперпараметрами и на других подвыборках обучающих данных. Кроме этого, авторы работы исследовали интерпретируемость фильтров промежуточных слоев сверточных сетей.

В дальнейшем было обнаружено ([Kurakin et al., 2016]($Adversarial examples in the physical world$)), что "обманывать" модель с помощью зашумленных изображений можно даже в том случае, когда зашумленное изображение фотографируется камерой (см. [видео](https://www.youtube.com/watch?v=zQ_uMenoBCk)). При этом разные модели удается обманывать одним и тем же паттерном шума, что позволяет осуществлять подобные атаки (с переменным успехом) даже тогда, когда используемая модель неизвестна.

В 2017 году изобрели совсем простой способ "обмануть" сверточную нейронную сеть: достаточно наклеить на объект стикер со специальным рисунком, и сеть перестанет правильно классифицировать объект ([Brown et al., 2017]($Adversarial Patch$)). Впрочем, такой стикер не универсален и будет работать только для определенных сетей, против которых он создавался.

<img src="assets/patch.jpg" width="550" align="center">

Обзор других многочисленных работ по состязательным атакам можно найти в [Akhtar and Mian, 2018]($Threat of Adversarial Attacks on Deep Learning in Computer Vision: A Survey$) и [Akhtar et al., 2021]($Advances in adversarial attacks and defenses in computer vision: A survey$). Проблемы состязательных атак и недостаточной out-of-distribution генерализации в компьютерном зрении, вероятно, связаны между собой. Человеческое зрение тоже подвержено такому типу атак, что доказывают оптические иллюзии.

### Проблема неконкретизации в ML-задачах

Рассмотрим еще одну работу, изучающую поведение ML-моделей за пределами обучающего распределения данных.

Для каждой задачи может существовать бесконечное количество разных обученных моделей с разными весами и вычислительными архитектурами, дающих примерно одинаковую точность на *обучающей выборке*. Это известная и центральная проблема машинного обучения. Но эта же проблема существует и в другом варианте: для каждой задачи может существовать бесконечное количество разных обученных моделей, дающих примерно одинаковую точность на *обучающем распределении данных*, из которого взята тестовая выборка.

В работе [D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$) эту проблему называют проблемой неконкретизации ML-задачи (*underspecification of ML-pipeline*). Например, если задачей является обучить систему распознавания изображений с помощью сверточной сети, то может существовать огромное множество разных сверточных сетей с разной архитектурой и инициализацией весов, дающих примерно одинаковую точность на тестовой выборке. При смене условий (выходе за пределы обучающего распределения) эти модели могут начать работать сильно по-разному.

В одном из экспериментов авторы обучили на ImageNet 50 сверточных нейронных сетей ResNet-50 ([He et al., 2015]($Deep Residual Learning for Image Recognition$)). На валидационной части ImageNet все сети давали примерно одинаковую точность 75.9% ± 0.1% (указано одно стандартное отклонение). Затем авторы протестировали сети на датасете ImageNet-С ([Hendrycks and Dietterich, 2019]($Benchmarking Neural Network Robustness to Common Corruptions and Perturbations$)), содержащем изображения с различными шумами и искажениями (out-of-distrubiton данные с точки зрения ImageNet). В результате на изображениях с пикселизацией сети давали точность 19.7% ± 2.4%, на изображениях с уменьшенной или увеличенной контрастностью сети давали точность 9.1% ± 0.8% и так далее. Аналогичная ситуация наблюдалась на сетях ResNet-101x3 BiT ([Kolesnikov et al., 2019]($Big Transfer BiT: General Visual Representation Learning$)).

<img src="assets/underspec.jpg" width="700" align="center">

Видно, что точность моделей не только стала намного хуже, но и увеличился разброс значений точности среди моделей (2.4% против 0.1%). Более того, отклонения от средней точности на разных типах искажений почти не коррелируют. Это означает, что если модель $A$ справляется лучше модели $B$ на изображениях с пикселизацией, то это не означает, что она будет лучше справляться на изображениях с измененной контракстностью.

Подобный эксперимент авторы повторили и с другими данными: на медицинских изображениях, где использовались изображения и 5 камер (первые 4 камеры - обучающее распределение, 5-я камера - out-of-distrubiton данные), и получили аналогичный результат. Таким образом, *поведение моделей, имеющих одинаковую точность на тестовой выборке из обучающего распределения, может быть существенно разным за пределами этого распределения*.

А ведь в описанных выше экспериментах речь шла о моделях лишь с разной инициализацией весов, что уж говорить о моделях с разной архитектурой? Авторы предполагают, что не только архитектура, но и, скажем, оптимизатор и стратегия управления learning rate может сильно влиять на свойства модели (помимо общей точности на обучающем распределении) и точность работы полученной модели на тех или иных примерах за пределами обучающего распределения.

### Выводы

1. ML-модели часто полагаются на "обходные" способы получения ответа (shortcuts), вызванные недостаточным разнообразием обучающего распределения данных и наличия в нем паразитных корреляций (это напоминает утечку данных). Такая модель лишь имитирует решение задачи, и поэтому может перестать корректно работать, если условия изменятся.
2. Следовательно, модели часто показывают низкую точность работы на таких данных, вероятность встретить которые в обучающем датасете была невелика: например, диван леопардовой расцветки, лицо с нанесенной краской, объекты в необычном окружении или текст, в котором используются нестереотипные словосочетания.
3. Модели, показывающие одинаковую среднюю точность при тестировании, могут работать существенно по-разному, что особенно проявляется за пределами распределения данных, на котором модели обучались и тестировались.
4. Системы компьютерного зрения (и не только) подвержены состязательным атакам, при которых внесение различных помех или добавление незначительных деталей может испортить работу модели. Человеческое зрение намного устойчивее к таким атакам.

Здесь еще можно вспомнить платформу Kaggle для проведения ML-соревнований. Часто считают, что рейтинг на Kaggle не полностью отражает уровень ML-специалиста из-за того, что реальный процесс ML-разработки содержит намного больше шагов, чем просто обучения модели. Однако при этом считается, что Kaggle хорошо развивает умение обучать качественные ML-модели. Исходя из рассмотренного выше можно усомниться и в этом факте. На Kaggle качество моделей как правило оценивается на том же распределении, из которого взята обучающая выборка, поэтому проблемы shortcut learning и устойчивость модели к сдвигу данных не играет особой роли. Кроме того, утечки данных (как разновидность сдвига данных) в реальной работе вредны, а на Kaggle скорее полезны.

## Способы решения проблем обобщения

Закончив раздел "кто виноват", перейдем к разделу "что делать". Вопрос в том, как изгнать "умного Ганса" из ML-моделей и обучить их в полной мере решать поставленную задачу. Рассмотрим сначала наиболее прямолинейные способы, и закончим исследовательскими гипотезами и обзором работ, посвященных разработкам новых архитектур и методов обучения.

### Стресс-тесты для диагностики работы модели

Авторы статей о shortcut learning ([Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$)) и неконкретизации ([D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$)) сходятся в том, что необходимо проверять не только оценивать общую точность модели на тестовой выборке, но и оценивать точность на отдельных типах примеров. В частности, авторы предлагают следующие типы тестов:

1. **Stratified evaluations.** Разбиение тестовой выборки на типы (например, по значению какого-то признака, по классу) и оценка точности модели отдельно на каждом типе примеров. Процитирую пример из [другого обзора](https://habr.com/ru/company/ods/blog/599573/):

> Пусть модель тестировалась на датасете, в котором 80% изображений были высокого качества (HQ), а применяться будет в условиях, когда, наоборот, 80% изображений будут низкого качества (LQ). Пусть мы сравниваем две модели: на HQ-изображениях точность первой модели лучше, чем второй, а на LQ-изображениях, наоборот, точность второй модели лучше, чем первой. Если при тестировании большая часть изображений были HQ, то мы сделаем вывод, что первая модель лучше, тогда как на самом деле лучше была бы вторая.

2. **Shifted evaluations.** Тестирование модели на всех типах данных, на которых желательна ее хорошая работа. Например, на зашумленных данных, на фотографиях с разных времен года и так далее.

3. **Contrastive evaluations.** Вместо рассчета средней метрики качества модели по какой-либо выборке, мы изучаем изменение метрики качества при изменении выборки (или одного примера). Например, поворачиваем фотографии на один и тот же угол и сравниваем метрику качества. В задачах обработки естественного языка [Ribeiro et al., 2020]($Beyond Accuracy: Behavioral Testing of NLP Models with CheckList$) предлагают целый ряд проверок, которые стоит проделать на обученных моделях.

### Доменная адаптация

Перечисленные способы помогут вывить проблемы в функционировании модели. После этого соответствующие примеры можно добавить в обучающие данные (или увеличить их процентное соотношение). Обучая модель на недостающих типах данных, мы превращаем out-of-distribution данные в in-distribution данные и повышаем качество работы модели на этих данных. Это наиболее прямолинейное решение проблемы сдвига данных называется доменной адаптацией (см. на [paperswithcode.com](https://paperswithcode.com/task/domain-adaptation). Для дообучения можно использовать и неразмеченные данные (см. обзор на Хабре: [часть 1](https://habr.com/ru/company/vk/blog/426803/), [часть 2](https://habr.com/ru/company/vk/blog/429966/)). Но такой способ не универсален. Мы можем не знать заранее всех свойств данных, с которыми будет работать модель, и не иметь возможности собрать обучающие данные.

Существует так называемая "проблема черного лебедя" ([*black swan*](https://en.wikipedia.org/wiki/Black_swan_theory)), когда ошибка на очень редком типе примеров может привести к значительным последствиям: например, автоматически управляемый автомобиль попадет в ДТП из-за сбоя в работе его систем компьютерного зрения. Есть много примеров, когда электромобили Tesla неправильно распознавали происходящее вокруг, например, [принимали луну за сфетфор](https://www.youtube.com/watch?v=7UF-S2czdCk). Насколько бы разнообразным ни был датасет, все равно может оставаться шанс, что мы что-то не учли и не добавили в него какой-то тип примеров.

Если задачей является обучение модели общего назначения для дальнейшего файн-тюнинга (например, предобучение языковой модели, сети для распознавания изображений, сети для распознавания речи или же мультимодальной сети, наподобие CLIP ([Radford et al., 2021]($Learning Transferable Visual Models From Natural Language Supervision$))), то скрытых bias'ов в данных и возможностей для shortcut learning неисчислимое множество, и невозможно устранить их все.

В целом, система с низким out-of-distribution обобщением производит впечатление не слишком "интеллектуальной", а доменная адаптация часто напоминает подстановку "костылей" под модель, не способную хорошо обобщаться. Нужно ли искать новые архитектуры и способы обучения, более устойчивые к сдвигу данных?

### Еще больше данных?

От себя могу предположить, что, как ни странно, еще большее увеличение объема обучающих данных может помочь. С увеличением объема данных почти всегда растет разнообразие данных (умозрительно можно предположить, что если объем растет линейно, то разнообразие растет логарифмически). А ведь разнообразие данных - это именно то, что нужно? Возможно, с какого-то момента, когда loss уже нельзя будет еще сильнее понизить простым запоминанием поверхностной статистики (т. е. с помощью shortcut'ов), процесс оптимизации будет стагнировать и будет вынужден начать искать другие пути понижения loss, в том числе через систематическую генерализацию.

Конечно эта лишь гипотеза. Но на мой взгляд важно иметь в виду, что зависимость качества обучения от размера модели и объема данных может быть необычной и непредсказуемой. Например, в [Nakkiran et al., 2019]($Deep Double Descent: Where Bigger Models and More Data Hurt$) (double descent) была показана необычная зависимость точности на валидации от размера модели, а в [Power et al., 2022]($Grokking: Generalization Beyond Overfitting on Small Algorithmic Datasets$) (grokking) от длительности обучения.

<img src="assets/unexpected.jpg" width="700" align="center">

### Более крупные или более эффективные модели?

Сейчас глубокое обучение в основном идет по пути экстенсивного развития, при котором улучшения достигаются увеличением объема обучающих данных и размера моделей ("stack more layers"). Разработке принципиально иных архитектур и способов обучения уделяется мало внимания и финансирования, что и понятно: в таких исследованиях нет никакой гарании успеха и получения работающего решения в обозримые сроки. Но как мы увидели выше, простое увеличение объема данных не всегда дает решение, обладающее желаемым уровнем обобщения. Об этом говорят авторы работы, посвященной графовым сетям ([Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$)):

> The question of how to build artificial systems which exhibit combinatorial generalization has been at the heart of AI since its origins, and was central to many structuredapproaches, including logic, grammars, classic planning, graphical models, causal reasoning, Bayesian nonparametrics, and probabilistic programming  ([Chomsky, 1957]($Syntactic structures$); [Nilsson and Fikes, 1970]($STRIPS: A New Approach to the Application of Theorem Proving to Problem Solving$); [Pearl, 1986]($Fusion, Propagation, and Structuring in Belief Networks$), [2009]($Causality: Models, Reasoning and Inference$); [Russell and Norvig, 2009]($Artificial Intelligence: A Modern Approach$); [Hjort et al., 2010]($Bayesian Nonparametrics$); [Goodman et al., 2012]($Church: a language for generative models$); [Ghahramani, 2015]($Probabilistic machine learning and artificial intelligence$)). ... A key reason why structured approaches were so vital to machine learning in previous eras was, in part, because *data and computing resources were expensive*, and the improved sample complexity afforded by structured approaches’ strong inductive biases was very valuable.

> In contrast with past approaches in AI, modern deep learning methods ([LeCun et al., 2015]($Deep Learning Nature Journal$);
[Schmidhuber, 2015]($Deep Learning in Neural Networks: An Overview$); [Goodfellow et al., 2016]($Deep Learning$)) often follow an "end-to-end" design philosophy which emphasizes minimal *a priori* representational and computational assumptions, and seeks to avoid explicit structure and "hand-engineering". This emphasis hasfit well with — and has perhaps been affirmed by — the current abundance of *cheap data and cheap computing resources*, which make trading off sample efficiency for more flexible learning a rational choice. ...

> Despite deep learning’s successes, however, important critiques ([Marcus, 2001]($The algebraic mind$); [Shalev-Shwartz et al., 2017]($Failures of Gradient-Based Deep Learning$); [Lake et al., 2017]($Building Machines That Learn and Think Like People$); [Lake and Baroni, 2018]($Generalization without systematicity: On the compositional skills of sequence-to-sequence recurrent networks$); [Marcus, 2018a]($Deep Learning: A Critical Appraisal$), [2018b]($Innateness, AlphaZero, and Artificial Intelligence$); [Pearl, 2018]($Theoretical Impediments to Machine Learning With Seven Sparks from the Causal Revolution$); [Yuille and Liu, 2018]($Deep Nets: What have they ever done for Vision?$)) have highlighted key challenges it faces in *complex language and scene understanding, reasoning about structured data, transferring learning beyond the training conditions, and learning from small amounts of experience*. These challenges demand combinatorial generalization, and so it is perhaps not surprising that an approach which eschews compositionality and explicit structure struggles to meet them.

Авторы работы, посвященной системам компьютерного зрения ([Yuille and Liu, 2018]($Deep Nets: What have they ever done for Vision?$)), также делают вывод о том, что больших данных может быть недостаточно, и нужно искать новые подходы к обучению и оценке качества моделей.

> We argue that Deep Nets in their current form are unlikely to be able to overcome the fundamental problem of computer vision, namely how to deal with the combinatorial explosion, caused by the enormous complexity of natural images, and obtain the rich understanding of visual scenes that the human visual achieves. We argue that this combinatorial explosion takes us into a regime where "big data is not enough" and where we need to rethink our methods for benchmarking performance and evaluating vision algorithms. ...  We question whether Deep Nets
will be sufficient to address these challenges and argue that methods that are compositional, generative, and combine signal and symbolic processing will be needed.

Рост результатов на различных бенчмарках, например SuperGLUE, тоже в основном обусловлен ростом размера моделей и количества обучающих данных. Но обучающие данные все же не бесконечны. Современные языковые модели и так обучаются почти на всех данных, какие удалось собрать со всего интернета.

В какой-то момент, скорее всего, потребуется прекратить рост размера моделей, объема датасетов и сосредоточиться на разработке совершенно иных подходов, стандартизировав наборы обучающих данных (как предлагает [Linzen, 2020]($How Can We Accelerate Progress Towards Human-like Linguistic Generalization?$)) для лучшего сравнения эффективности подходов.

<img src="assets/nostack.jpg" width="300" align="center">

### Модификации способов обучения языковых моделей

Современные языковые модели предобучаются, например, на следующих задачах:

1. Предсказание токенов (т. е. слов или их частей), закрытых маской (masked language modeling, BERT, [Devlin et al., 2018]($BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding$)). Иногда используют модификацию этой задачи, в которой закрываются маской словосочетания, представляющие собой целостные сущности (knowledge masking, ERNIE, [Sun et al., 2019]($ERNIE: Enhanced Representation through Knowledge Integration$)).

2. Предсказания следующего токена в тексте (generative pre-training, GPT, [Radford et al., 2018]($Improving Language Understanding by Generative Pre-Training$)). При этом на архитектуру модели накладывается ограничение: $i$-й входной токен не должен влиять на выходные токены с индексами меньше $i$.

3. Предсказание отдельных токенов в тексте, полученном конкатенацией предложения из энциклопедии и информации из графа знаний (universal knowledge-text prediction, ERNIE 3.0, [Sun et al., 2021]($ERNIE 3.0: Large-scale Knowledge Enhanced Pre-training for Language Understanding and Generation$)). Кроме этой задачи, модель ERNIE 3.0 также обучается на задачах generative pre-training и knowledge masked language modeling.

4. Наборы sequence-to-sequence задач, генерируемых в self-supervised режиме, в том числе аналоги задачи masked language modeling (T5, [Raffel et al., 2019]($Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer$), см. figure 2, table 3).

Чтобы лучше понять свойства этих задач, можно попробовать порешать их самостоятельно. Я сделал [Colab-ноутбук](https://colab.research.google.com/drive/17p6DD8kcpkp-nBRcIFfD7jAmAxmN7rSN?usp=sharing) для генерации случайных обучающих примеров в задаче masked language modeling (MLM). Из примеров можно видеть следующее:

1. Существенная часть замаскированных токенов определяется тривиально

2. Во многих случаях предскать замаскированные токены исходя из контекста невозможно, а значит для решения задачи на большом корпусе текста нужно запоминать множество различных фактов: сюжеты литературных произведений, историю, политику и так далее, а также множество статистики совместной встречаемости слов в тексте (shortcuts). Даже выучив множество фактов и статистики, решить задачу MLM можно далеко не всегда.

3. Лишь изредка возможно предсказать замаскированный токен на основании контекста и логики, и чаще всего эта логика достаточно простая. Исчезающе редки примеры, где для предсказания нужна нетривиальная логика, которую нельзя имитировать запоминанием фактов и статистики.

Следовательно, приоритетом для снижения loss на задаче MLM является запоминание фактов и статистики. Способность рассуждать и делать выводы (отображая слова в некую модель мира и обратно, как это делает человек) для снижения loss является лишь второстепенным фактором, при этом выучить эту способность должно быть намного сложнее, чем запомнить факты.

Получается, что мы обучаем сеть на такой задаче, которую нельзя хорошо решить, обладая одной лишь логикой и пониманием смысла общеупотребимых слов, но можно неплохо решить с помощью запоминания огромного объема фактов и shortcut'ов. Обученная таким способом модель действительно может оказаться способной генерировать правдоподобные тексты, но лишь "вспоминая", что когда-то видела что-то похожее. Таким образом, модель долго и старательно обучается, но вовсе не тому, чего мы от нее хотим.

<img src="assets/holmes.jpg" width="650" align="center">

Отсюда можно сделать вывод о том, что обучение сетей, которые действительно понимают смысл текста в том смысле, в каком его понимает человек (вместо использования shortcut'ов для имитации понимания текста) будет затруднительным, если принципиально не менять задачу предобучения.

### Архитектура моделей и структура данных

Модель эффективно обобщается, если ее структура соответствует структуре моделируемых данных. По этой причине на выборке, сгенерированной случайно инициализированной нейронной сетью, хорошо обобщается другая нейронная сеть ([Gorishniy et al., 2021]($Revisiting Deep Learning Models for Tabular Data$)), на выборке, сгенерированной случайным решающим деревом, хорошо обобщаются другие деревья, сверточные сети хорошо обобщаются на текстурах и составленных из них изображениях ([Ulyanov et al., 2017]($Deep Image Prior$)), а AlphaFold 2 хорошо обобщается на трехмерных структурах белков ([Jumper et al., 2021]($Highly accurate protein structure prediction with AlphaFold$)). Какова же структура текстов и изображений? Можно назвать два основных уровня:

*Уровень 1. Низкоуровневая локальная структура*. В изображениях это набор локальных признаков (текстуры и границы), в текстах это буквы и морфология слов. Также сюда можно отнести уровень звукового/визуального восприятия текста и интонацию.

*Уровень 2. Высокоуровневая иерархическая разреженная структура*. В текстах эта структура начинается с синтаксиса, следующим уровнем явлется связь между предложениями, абзацами и более крупными частями текста. В изображениях обычно тоже есть некая иерархическая структура, соответствующая объектам и их частям в трехмерном пространстве на разном расстоянии. При этом между элементами иерархической структуры имеется разреженный граф смысловых и причинно-следственных взаимосвязей. Например, рассуждая о человеке на фотографии, мы никак не будем учитывать цвет стоящего неподалеку автомобиля. Разреженность помогает не обращать внимание на паразитные корреляции и избегать shortcut learning'а.

Кроме того, когда человек воспринимает фотографию, иерархическая структура изображения отображается в модель мира, и человек сознательно или подсознательно выводы о том, логично ли изображенное на фотографии, в каком контексте и зачем оно сделано и каково развитие событий в момент съемки. С текстом все в целом аналогично.

Судя по всему, для исправления проблемы shortcut learning модель должна реализовывать именно такой способ обработки изображения или текста, то есть уметь представлять входные данные (изображение или текст) как иерархическую комбинацию составляющих частей с разреженными взаимосвязями, что соответствует уровню 2, и быть способной корректно интерпретировать те же части в необычных комбинациях. По сравнению с абстрактным свойством out-of-distribution обобщения, это более конкретное желаемое свойство называют *систематическим, композиционным, или комбинаторным обобщением* ([Bahdanau et al., 2018]($Systematic Generalization: What Is Required and Can It Be Learned?$); [Furrer et al., 2020]($Compositional Generalization in Semantic Parsing: Pre-training vs. Specialized Architectures$)). Достижение такого уровня обобщения остается открытой проблемой машинного обучения.

<iframe width="560" height="315" src="https://www.youtube.com/embed/T3sxeTgT4qc" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

> If you think about child learning, for example, their world is changing over time, their body is changing over time, and so we need systems that are going to be able to handle those changes and do things like continual learning, life-long learning and so on. This has been a long-standing goal for machine learning, but I think we haven't yet built the solutions to this. And one of the crucial elements in order to be successful in this ... is introducing more forms of compositionality. It means being able to learn from some finite sets of combinations about a much larger set of combinations. (Yoshua Bengio, NeurIPS 2019)

Здесь часто вспоминают [символьные подходы](https://en.wikipedia.org/wiki/Symbolic_artificial_intelligence), с которых и начинались исследования ИИ в XX веке. По структуре такие системы, кажется, обладают всем необходимым: они могут работать с объектами и их частями, то есть с интерпретируемыми промежуточными данными, работать иерархически, реализовывать сложные цепочки рассужедний, а вычисления внутри них могут затрагивать лишь немногие объекты и взаимосвязи, что означает разреженность. Непонятно лишь одно: как их обучать, чтобы они могли быть сложными и достаточно гибкими одновременно.

Например, пусть некая подсистема решает, какую из операций из набора $\{F_1, F_2, F_3\}$ применить к данным $x$. Для этого она выдает вероятности $P(F_1), P(F_2), P(F_3)$ и выбирает операцию с наибольшей вероятностью. Допустим, на текущем примере мы выбрали операцию $A = \text{argmax}_i P(F_i)$, и в итоге получили значение функции потерь $L$. Производные $\partial L / \partial P(F_i)$ равны нулю, поэтому такую систему не получится обучить обычным градиентным спуском. Можно попробовать рассчитать ответ, используя по очереди все $F_i$, и снизить вероятности $P(F_i)$ для тех $F_i$, на которых loss получился выше, чем на других. Но если граф принятия решений содержит не одну, а много операций принятия решения ($\text{argmax}$), тогда придется перебирать очень много разных вариантов последовательностей решений. Аналогичная проблема возникает, если мы выбираем не операцию, а один или несколько элементов среди доступных данных (*hard attention*).

### Направления исследований

В машинном обучении каждый день публикуется множество научных статей. Что-то модно, что-то вышло из моды, какие-то работы еще ждут признания, а какие-то так и остались незамеченными. Приведенный ниже список статей не претендует на полноту и описывает лишь некоторые из современных направлений исследований.

В перечисленных ниже работах изучаются проблемы современных систем компьютерного зрения. В некоторых из них также предлагаются различные модификации процесса обучения и способа оценки качества моделей, в том числе с помощью *meta-learning*.

- [Szegedy et al., 2013]($Intriguing properties of neural networks$). *Intriguing properties of neural networks.*
- [Peng et al., 2017]($Zero-Shot Deep Domain Adaptation$). *Zero-Shot Deep Domain Adaptation.*
- [Yuille and Liu, 2018]($Deep Nets: What have they ever done for Vision?$) *Deep Nets: What have they ever done for Vision?*
- [Li et al., 2017]($Learning to Generalize: Meta-Learning for Domain Generalization$). *Learning to Generalize: Meta-Learning for Domain Generalization.*
- [Beery et al., 2018]($Recognition in Terra Incognita$). *Recognition in Terra Incognita.*
- [Baker et al., 2018]($Deep convolutional networks do not classify based on global object shape$). *Deep convolutional networks do not classify based on global object shape.*
- [Balaji et al., 2018]($MetaReg: Towards Domain Generalization using Meta-Regularization$). *MetaReg: Towards Domain Generalization using Meta-Regularization.*
- [Hendrycks et al., 2019]($Natural Adversarial Examples$). *Natural Adversarial Examples.*
- [Brendel and Bethge, 2019]($Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet$). *Approximating CNNs with Bag-of-local-Features models works surprisingly well on ImageNet.*
- [Lapuschkin et al., 2019]($Unmasking Clever Hans Predictors and Assessing What Machines Really Learn$). *Unmasking Clever Hans Predictors and Assessing What Machines Really Learn*.
- [Wang et al., 2019]($High Frequency Component Helps Explain the Generalization of CNN$). *High Frequency Component Helps Explain the Generalization of CNN.*
- [Ilyas et al., 2019]($Adversarial Examples Are Not Bugs, They Are Features$). *Adversarial Examples Are Not Bugs, They Are Features*.
- [Bucher et al., 2019]($Zero-Shot Semantic Segmentation$). *Zero-Shot Semantic Segmentation.*
- [Taori et al., 2020]($Measuring Robustness to Natural Distribution Shifts in Image Classification$). *Measuring Robustness to Natural Distribution Shifts in Image Classification*.
- [Djolonga et al., 2020]($On Robustness and Transferability of Convolutional Neural Networks$). *On Robustness and Transferability of Convolutional Neural Networks*.
- [Mahajan et al., 2020]($Domain Generalization using Causal Matching$). *Domain Generalization using Causal Matching*.
- [Zhou et al., 2020]($Deep Domain-Adversarial Image Generation for Domain Generalisation$). *Deep Domain-Adversarial Image Generation for Domain Generalisation*.
- [Fabbrizzi et al., 2021]($A Survey on Bias in Visual Datasets$). *A Survey on Bias in Visual Datasets*.
- [Shu et al., 2021]($Open Domain Generalization with Domain-Augmented Meta-Learning$). *Open Domain Generalization with Domain-Augmented Meta-Learning.*
- [Zhou et al., 2021]($MixStyle Neural Networks for Domain Generalization and Adaptation$). *MixStyle Neural Networks for Domain Generalization and Adaptation.*

Проблемы современных языковых моделей, проблемы данных, на которых они обучаются, и некоторые способы улучшения моделей и данных описаны в следующих работах:
- [Gururangan et al., 2018]($Annotation Artifacts in Natural Language Inference Data$). *Annotation Artifacts in Natural Language Inference Data*.
- [Kaushik and Lipton, 2018]($How Much Reading Does Reading Comprehension Require? A Critical Investigation of Popular Benchmarks$). *How Much Reading Does Reading Comprehension Require? A Critical Investigation of Popular Benchmarks*.
- [Jawahar et al., 2019]($What does BERT learn about the structure of language?$). *What does BERT learn about the structure of language?*.
- [Goldberg, 2019]($Assessing BERTs Syntactic Abilities$). *Assessing BERT's Syntactic Abilities*.
- [McCoy et al., 2019]($Right for the Wrong Reasons: Diagnosing Syntactic Heuristics in NLI$). *Right for the Wrong Reasons: Diagnosing Syntactic Heuristics in NLI*.
- [Zellers et al., 2019]($HellaSwag: Can a Machine Really Finish Your Sentence?$). *HellaSwag: Can a Machine Really Finish Your Sentence?*.
- [Lin et al., 2019]($Open Sesame: Getting Inside BERTs Linguistic Knowledge$). *Open Sesame: Getting Inside BERT's Linguistic Knowledge*.
- [Kovaleva et al., 2019]($Revealing the Dark Secrets of BERT$). *Revealing the Dark Secrets of BERT*.
- [Kavumba et al., 2019]($When Choosing Plausible Alternatives, Clever Hans can be Clever$). *When Choosing Plausible Alternatives, Clever Hans can be Clever*.
- [McCoy et al., 2019]($BERTs of a feather do not generalize together: Large variability in generalization across models with similar test set performance$). *BERTs of a feather do not generalize together: Large variability in generalization across models with similar test set performance*.
- [Keysers et al., 2019]($Measuring Compositional Generalization: A Comprehensive Method on Realistic Data$). *Measuring Compositional Generalization: A Comprehensive Method on Realistic Data*.
- [Rogers, 2020]($A Primer in BERTology: What we know about how BERT works$). *A Primer in BERTology: What we know about how BERT works*.
- [Goodwin et al., 2020]($Probing Linguistic Systematicity$). *Probing Linguistic Systematicity*.
- [Ribeiro et al., 2020]($Beyond Accuracy: Behavioral Testing of NLP models with CheckList$). *Beyond Accuracy: Behavioral Testing of NLP models with CheckList*.
- [Linzen, 2020]($How Can We Accelerate Progress Towards Human-like Linguistic Generalization?$). *How Can We Accelerate Progress Towards Human-like Linguistic Generalization?*.
- [Bender and Koller, 2020]($Climbing towards NLU: On Meaning, Form, and Understanding in the Age of Data$). *Climbing towards NLU: On Meaning, Form, and Understanding in the Age of Data*.
- [Yu and Ettinger, 2020]($Assessing Phrasal Representation and Composition in Transformers$). *Assessing Phrasal Representation and Composition in Transformers*.
- [Liu et al., 2021]($GPT Understands, Too$). *GPT Understands, Too*.
- [Du et al., 2021]($Towards Interpreting and Mitigating Shortcut Learning Behavior of NLU Models$). *Towards Interpreting and Mitigating Shortcut Learning Behavior of NLU Models*.
- [Shin et al., 2021]($Constrained Language Models Yield Few-Shot Semantic Parsers$). *Constrained Language Models Yield Few-Shot Semantic Parsers*.
- [Yanaka et al., 2021]($SyGNS: A Systematic Generalization Testbed Based on Natural Language Semantics$). *SyGNS: A Systematic Generalization Testbed Based on Natural Language Semantics*.
- [Sanh et al., 2021]($Multitask Prompted Training Enables Zero-Shot Task Generalization$). *Multitask Prompted Training Enables Zero-Shot Task Generalization*.
- [Cobbe et al., 2021]($Training Verifiers to Solve Math Word Problems$). *Training Verifiers to Solve Math Word Problems*.
- [Shavrina and Malykh, 2021]($How not to Lie with a Benchmark: Rearranging NLP Leaderboards$). *How not to Lie with a Benchmark: Rearranging NLP Leaderboards*.
- [Veres, 2021]($Language Models are not Models of Language$). *Language Models are not Models of Language*.
- [Ouyang et al., 2022]($Training language models to follow instructions with human feedback$). *Training language models to follow instructions with human feedback*.
- [Polu et al., 2022]($Formal Mathematics Statement Curriculum Learning$). *Formal Mathematics Statement Curriculum Learning*.

Исследователи давно ищут возможность обучить модели систематическому обобщению, понимаю понятий части и целого и оперированию абстрактными концепциями. Некоторые из последних работ по этой теме:

- [Lake et al., 2015]($Human-level concept learning through probabilistic program induction$). *Human-level concept learning through probabilistic program induction*.
- [Bengio, 2017]($The Consciousness Prior$). *The Consciousness Prior*.
- [Sabour et al., 2017]($Dynamic Routing Between Capsules$) *Dynamic Routing Between Capsules.*
- [Evans and Grefenstette, 2017]($Learning Explanatory Rules from Noisy Data$). *Learning Explanatory Rules from Noisy Data*.
- [Bahdanau et al., 2018]($Systematic Generalization: What Is Required and Can It Be Learned?$). *Systematic Generalization: What Is Required and Can It Be Learned?*.
- [Battaglia et al., 2018]($Relational inductive biases, deep learning, and graph networks$). *Relational inductive biases, deep learning, and graph networks*.
- [Vlastelica et al., 2021]($Neuro-algorithmic Policies enable Fast Combinatorial Generalization$). *Neuro-algorithmic Policies enable Fast Combinatorial Generalization*.
- [Hinton, 2021]($How to represent part-whole hierarchies in a neural network$). *How to represent part-whole hierarchies in a neural network*.

Высокая способность к обобщению также требуется при обучении агентов, так как окружение может быть очень разнообразным и меняться от действий самих агентов, поэтому важным является способность корректно функционировать в новых условиях, не встречавшихся раньше.
- [Gupta et al., 2021]($Embodied Intelligence via Learning and Evolution$). *Embodied Intelligence via Learning and Evolution*.
- [Hazra et al., 2021]($Zero-Shot Generalization using Intrinsically Motivated Compositional Emergent Protocols$). *Zero-Shot Generalization using Intrinsically Motivated Compositional Emergent Protocols*.
- [Jang et al., 2021]($BC-Z: Zero-Shot Task Generalization with Robotic Imitation Learning$). *BC-Z: Zero-Shot Task Generalization with Robotic Imitation Learning*.

Мнение многих исследователей сходится в том, что для создания ML-систем следующего поколения, способных исправить важные недостатки текущих систем, нужно пробовать объединять нейронные сети и символьные вычисления:

- [Besold et al., 2017]($Neural-Symbolic Learning and Reasoning: A Survey and Interpretation$). *Neural-Symbolic Learning and Reasoning: A Survey and Interpretation*.
- [Mao et al., 2019]($The Neuro-Symbolic Concept Learner: Interpreting Scenes, Words, and Sentences From Natural Supervision$). *The Neuro-Symbolic Concept Learner: Interpreting Scenes, Words, and Sentences From Natural Supervision*.
- [Vankov and Bowers, 2019]($Training neural networks to encode symbols enables combinatorial generalization$). *Training neural networks to encode symbols enables combinatorial generalization*.
- [Latapie et al., 2021]($Neurosymbolic Systems of Perception Cognition: The Role of Attention$). *Neurosymbolic Systems of Perception & Cognition: The Role of Attention*.
- [Niepert et al., 2021]($Implicit MLE: Backpropagating Through Discrete Exponential Family Distributions$). *Implicit MLE: Backpropagating Through Discrete Exponential Family Distributions*.

Одним из многообещающих подходов к повышению обобщающей способности моделей считается meta-learning ("learning-to-learn"):
- [Hospedales et al., 2020]($Meta-Learning in Neural Networks: A Survey$). *Meta-Learning in Neural Networks: A Survey*.

Много работ посвящено попыткам улучшения [трансформера]($Attention Is All You Need$), а также попыткам перенести его свойства (*inductive biases*) на другие архитектуры или найти более простые альтернативы. Трансформер можно рассматривать как графовую нейронную сеть, в которой вершины хранят информацию, а ребра не хранят информации. Однако недавно были опубликованы работы, в которых предлагаются модификации трансформера с хранением информации в ребрах (подобный подход уже использовался, например, в [AlphaFold 2]($Highly accurate protein structure prediction with AlphaFold$)). Также была обнаружена интересная взаимосвязь трансформера с сетями Хопфилда.
- [Kerg et al., 2019]($Untangling tradeoffs between recurrence and self-attention in neural networks$). *Untangling tradeoffs between recurrence and self-attention in neural networks*.
- [Gao et al., 2020]($Systematic Generalization on gSCAN with Language Conditioned Embedding$). *Systematic Generalization on gSCAN with Language Conditioned Embedding*.
- [Ramsauer et al., 2020]($Hopfield Networks is All You Need$). *Hopfield Networks is All You Need*.
- [Bello, 2021]($LambdaNetworks: Modeling Long-Range Interactions Without Attention$). *LambdaNetworks: Modeling Long-Range Interactions Without Attention*.
- [Pan et al., 2021]($On the Integration of Self-Attention and Convolution$). *On the Integration of Self-Attention and Convolution*.
- [Liu et al., 2021]($Pay Attention to MLPs$). *Pay Attention to MLPs*.
- [Lee-Thorp et al., 2021]($FNet: Mixing Tokens with Fourier Transforms$). *FNet: Mixing Tokens with Fourier Transforms*.
- [Xu et al., 2021]($CDTrans: Cross-domain Transformer for Unsupervised Domain Adaptation$). *CDTrans: Cross-domain Transformer for Unsupervised Domain Adaptation*.
- [Nawrot et al., 2021]($Hierarchical Transformers Are More Efficient Language Models$). *Hierarchical Transformers Are More Efficient Language Models*.
- [Bergen et al., 2021]($Systematic Generalization with Edge Transformers$). *Systematic Generalization with Edge Transformers*.
- [Hussain et al., 2021]($Edge-augmented Graph Transformers: Global Self-attention is Enough for Graphs$). *Edge-augmented Graph Transformers: Global Self-attention is Enough for Graphs*.
- [Wang et al., 2022]($When Shift Operation Meets Vision Transformer: An Extremely Simple Alternative to Attention Mechanism$). When Shift Operation Meets Vision Transformer: An Extremely Simple Alternative to Attention Mechanism.


Если модель встречается с необычным примером и не может выдать на нем корректный ответ, то можно по крайней мере научить модель сомневаться, то есть оценивать уверенность в предсказании. Это может помочь не только в случае сдвига данных, но и на тех примерах, на которых точечная оценка условного распределения $P(Y|X)$ даст высокую ожидаемую ошибку (то есть ответ нельзя с уверенностью предсказать из исходных данных). В этом контексте часто упоминают байесовские методы, ансамблирование и генеративные модели. Некоторые из работ по теме байесовских методов и оценки уверенности в предсказаниях:

- [Wang and Vasconcelos, 2018]($Towards Realistic Predictors$). *Towards Realistic Predictors*.
- [Maddox et al., 2019]($A Simple Baseline for Bayesian Uncertainty in Deep Learning$). *A Simple Baseline for Bayesian Uncertainty in Deep Learning*.
- [Nalisnick et al., 2019]($Detecting Out-of-Distribution Inputs to Deep Generative Models Using Typicality$). *Detecting Out-of-Distribution Inputs to Deep Generative Models Using Typicality*.
- [Ren et al., 2019]($Likelihood Ratios for Out-of-Distribution Detection$). *Likelihood Ratios for Out-of-Distribution Detection*.
- [Jospin et al., 2020]($Hands-on Bayesian Neural Networks - a Tutorial for Deep Learning Users$). *Hands-on Bayesian Neural Networks - a Tutorial for Deep Learning Users*.
- [Lakshminarayanan, 2020]($Uncertainty Out-of-Distribution Robustness in Deep Learning$). *Uncertainty & Out-of-Distribution Robustness in Deep Learning*.
- [Yang et al., 2021]($Generalized Out-of-Distribution Detection: A Survey$). *Generalized Out-of-Distribution Detection: A Survey*.
- [Henning et al., 2021]($Are Bayesian neural networks intrinsically good at out-of-distribution detection?$). *Are Bayesian neural networks intrinsically good at out-of-distribution detection?*
- [Huyen, 2022]($Data Distribution Shifts and Monitoring$). *Data Distribution Shifts and Monitoring*.

Еще несколько общих статей о проблемах современного машинного обучения, паразитных корреляциях в данных и подходах к повышению обобщающей способности (некоторые из них подробно разбирались в этом обзоре):

- [Tommasi et al., 2015]($A Deeper Look at Dataset Bias$). *A Deeper Look at Dataset Bias*.
- [Ross et al., 2017]($Right for the Right Reasons: Training Differentiable Models by Constraining their Explanations$). *Right for the Right Reasons: Training Differentiable Models by Constraining their Explanations*.
- [Shalev-Shwartz et al., 2017]($Failures of Gradient-Based Deep Learning$). *Failures of Gradient-Based Deep Learning*.
- [Marcus, 2018]($Deep Learning: A Critical Appraisal$). *Deep Learning: A Critical Appraisal*.
- [Mehrabi et al., 2019]($A Survey on Bias and Fairness in Machine Learning$). *A Survey on Bias and Fairness in Machine Learning*.
- [Geirhos et al., 2020]($Shortcut Learning in Deep Neural Networks$). *Shortcut Learning in Deep Neural Networks*.
- [D'Amour et al., 2020]($Underspecification Presents Challenges for Credibility in Modern Machine Learning$). *Underspecification Presents Challenges for Credibility in Modern Machine Learning*.
- [Xu et al., 2020]($How Neural Networks Extrapolate: From Feedforward to Graph Neural Networks$). *How Neural Networks Extrapolate: From Feedforward to Graph Neural Networks*.
- [Balestriero et al., 2021]($Learning in High Dimension Always Amounts to Extrapolation$). *Learning in High Dimension Always Amounts to Extrapolation*.
- [Tetteh et al., 2021]($Multi-Domain Balanced Sampling Improves Out-of-Distribution Generalization of Chest X-ray Pathology Prediction Models$). *Multi-Domain Balanced Sampling Improves Out-of-Distribution Generalization of Chest X-ray Pathology Prediction Models*.
- [Shen et al., 2021]($Towards Out-Of-Distribution Generalization: A Survey$). *Towards Out-Of-Distribution Generalization: A Survey*.

Многие исследователи считают, что важно не только обучить модели выявнять статистические зависимости, но и находить в данных причинно-следственные связи. Имея лишь выборку из распределения данных, это не всегда можно сделать. Например, пусть мы имеем набор фотографий с неизвестной планеты, и на некоторых фотографиях есть огонь и есть неопознанные существа, при этом появление существ и огня на фотографиях сильно коррелирует. Что является причиной, а что следствием? Разводят ли существа огонь сами, приходят ли к уже появившемуся огню, или же оба явления имеют общую причину (прилетает звездолет, высаживает существ и оставляет на земле огонь после взлета)? Это практически невозможно определить из фотографий (или же очень сложно, по косвенным признакам). Поиска причинно-следственных связей является более сложной задачей, чем поиск статистических зависимостей. Следующие книги и статьи посвящены именно этой теме:

- [Pearl, 2009]($Causality: Models, Reasoning and Inference$). *Causality: Models, Reasoning and Inference, 2nd edition*.
- [Schölkopf, 2019]($Causality for Machine Learning$). *Causality for Machine Learning*.
- [Arjovsky et al., 2019]($Invariant Risk Minimization$). *Invariant Risk Minimization*.
- [Ahuja et al., 2021]($Invariance Principle Meets Information Bottleneck for OOD Generalization$). *Invariance Principle Meets Information Bottleneck for OOD Generalization*.

Отдельно можно отметить работу группы Йошуа Бенжио в институте MILA (Канада):

- [Bengio et al., 2019]($A Meta-Transfer Objective for Learning to Disentangle Causal Mechanisms$). *A Meta-Transfer Objective for Learning to Disentangle Causal Mechanisms*.
- [Goyal et al., 2019]($Recurrent Independent Mechanisms$). *Recurrent Independent Mechanisms*.
- [Ke et al., 2019]($Learning Neural Causal Models from Unknown Interventions$). *Learning Neural Causal Models from Unknown Interventions*.
- [Ahmed et al., 2020]($Systematic generalisation with group invariant predictions$). *Systematic generalisation with group invariant predictions*.
- [Goyal and Bengio, 2020]($Inductive Biases for Deep Learning of Higher-Level Cognition$). *Inductive Biases for Deep Learning of Higher-Level Cognition*.
- [Schölkopf et al., 2021]($Towards Causal Representation Learning$). *Towards Causal Representation Learning*.
- [Liu et al., 2021]($Discrete-Valued Neural Communication$). *Discrete-Valued Neural Communication*.
- [Rahaman et al., 2021]($Dynamic Inference with Neural Interpreters$). *Dynamic Inference with Neural Interpreters*.

Различные модальности (изображения, текст и др.) могут дополнять друг друга  и формировать более целостную картину мира, чем каждая модельность по отдельности, поэтому мультимодальное обучение кажется многообещающим. В последние пару лет в этой области были достигнуты впечатляющие результаты.
- [Ramesh et al., 2021]($Zero-Shot Text-to-Image Generation$). *Zero-Shot Text-to-Image Generation*.
- [Radford et al., 2021]($Learning Transferable Visual Models From Natural Language Supervision$). *Learning Transferable Visual Models From Natural Language Supervision*.
- [Goh et al., 2021]($Multimodal Neurons in Artificial Neural Networks$). *Multimodal Neurons in Artificial Neural Networks*.
- [Singh et al., 2021]($Illiterate DALL-E Learns to Compose$). *Illiterate DALL-E Learns to Compose*.

Одним из возможных путей к созданию ML-систем следующего поколения может стать обучение систем, способных анализировать видео и предсказывать следующие события (далее чем несколько кадров), в том числе в текстовом виде. Такие системы могут быть способы отличать статистические зависимости от причинно-следственных связей. Та же задача возникает при обучении агентов, которым необходимо предсказывать последствия своих и чужих действий. Агента может заменить на наблюдателя, задача которого - лишь предсказывать послетствия чужий действий. Можно искать и другие способы создания самообучающейся модели мира, какая существует, например, в сознании человека.

- [Ha and Schmidhuber, 2018]($World Models$). *World Models*.
- [Lotter et al., 2018]($A neural network trained to predict future video frames mimics critical properties of biological neuronal responses and perception$). *A neural network trained to predict future video frames mimics critical properties of biological neuronal responses and perception.*
- [Zhang and Tao, 2019]($Slow Feature Analysis for Human Action Recognition$). *Slow Feature Analysis for Human Action Recognition.*
- [Xu et al., 2020]($Video Prediction via Example Guidance$). *Video Prediction via Example Guidance*.
- [Lee et al., 2021]($Revisiting Hierarchical Approach for Persistent Long-Term Video Prediction$). *Revisiting Hierarchical Approach for Persistent Long-Term Video Prediction*.
- [Wu et al., 2021]($Generative Video Transformer: Can Objects be the Words?$). *Generative Video Transformer: Can Objects be the Words?*.

### Общий искусственный интеллект

Исследователей всегда интересовала тема общего искусственного интеллекта и создания систем, которые мыслят как люди. Проблема здесь начинается с того, что разные люди по-разному определяют, что же конкретно это означает. Как мне кажется, здесь может помочь введение более узких терминов с более конкретными определениями, вместо расплывчатых и спорных понятий "общего ИИ" и "сильного ИИ". Если разные люди хотят предложить разные определения, то пусть придумают для них разные термины.

Например, можно предложить такое определение: *полнофункциональный искусственный интеллект* - это система, способная решать все задачи, которые может решать человек. Если больше не останется таких типов задач, которые может решить практически любой человек, но не может решить ИИ (качество решения оценивается голосованием людей), то тогда можно будет считать, что создан полнофункциональный ИИ, согласно его определению. Конечно, не обязательно требовать от ИИ выполнения механических движений или, скажем, мимики лица. Поэтому можно предложить более узкое понятие: *полнофункциональный текстовый искусственный интеллект* - это система, способная решать все задачи, которые может решать человек в текстовом виде, будь от по ту сторону монитора и имея достаточно времени. Конечно, и такое определение может породить споры, но все же оно несколько более конкретно.

Я никоим образом не являюсь специалистом по вопросу общего ИИ, но как автор обзора выскажу мнение по поводу его достижения. Меня привлекает идея постепенного улучшения, когда на каждом шаге ищутся недостатки систем текущего поколения и находятся способы их преодоления для создания систем следующего поколения. Можно провести аналогию с градиентным спуском или методом отжига: мы постепенно улучшаем решение, не пытаясь все выбросить и начать с нуля, и периодически пробуем вредрить какие-то принципиальные изменения. Тогда на текущем этапе развития машинного обучения нужно сосредоточиться на решении проблем обобщения, которым и был посвящен этот обзор.

Приведу также несколько работ, в которых рассматривается вопрос общего искусственного интеллекта и сопоставления биологического интеллекта с текущими ML-системами.

- [Marcus, 2001]($The algebraic mind$). *The Algebraic Mind: Integrating Connectionism and Cognitive Science*.
- [Marcus, 2004]($The Birth of the Mind: How a Tiny Number of Genes Creates The Complexities of Human Thought$). *The Birth of the Mind: How a Tiny Number of Genes Creates The Complexities of Human Thought*.
- [Griffiths et al., 2010]($Probabilistic models of cognition: exploring representations and inductive biases$). *Probabilistic models of cognition: exploring representations and inductive biases*.
- [Lake et al., 2016]($Building machines that learn and think like people$). *Building machines that learn and think like people*.
- [Chollet, 2019]($On the Measure of Intelligence$). *On the Measure of Intelligence*.
- [Marcus, 2020]($The Next Decade in AI: Four Steps Towards Robust Artificial Intelligence$). *The Next Decade in AI: Four Steps Towards Robust Artificial Intelligence*.
- [Stoop, 2021]($Note on the Reliability of Biological vs. Artificial Neural Networks$). *Note on the Reliability of Biological vs. Artificial Neural Networks*.
- [Fjelland, 2020]($Why general artificial intelligence will not be realized$). *Why general artificial intelligence will not be realized*.
- [Isomura et al., 2022]($Canonical neural networks perform active inference$). *Canonical neural networks perform active inference*.

## Заключение

Как мне кажется, проблемы содержатся не только в алгоритмах, но и в человеческом сознании. Людям *неинтересно* изучать и исследовать. Чтение статей, изучение информации воспринимается лишь как скучная необходимость, сложные вопросы и исследовательские задачи не занимают людей, а интересует в основном лиш заработок. Иногда [говорят](https://youtu.be/gGce2JDbJAQ?t=296), что для исследовательской работы нужен особый склад ума, но мне кажется, что в первую очередь нужен именно интерес к исследованиям. Его отсутствие у многих - это, на мой взгляд, проблема школьного и дошкольного образования, так как интерес к исследованиям является продуктом усилий учителей и родителей.

Еще одна сложность в том что количество научных статей и книг в области исследовательского ML стремительно растет. На конференции NeurIPS 2021 было опубликовано [более 2300 статей](https://papers.nips.cc/paper/2021) (больше, чем за 2010-2015 годы вместе взятые): одни только аннотации заняли бы целый том. Возникает острая потребность в систематизации информации. Читать статьи в оригинале долго и тяжело по нескольким причинам.

1. Каждая статья содержит обзор других работ, и если вы читаете, скажем, 20 статей по одной и той же области, то эти обзоры во многом дублируют друг друга и отбивают интерес к чтению.
2. Материал статьи может быть изложен не лучшим образом, непонятно или с повторениями. Сейчас не принято, как раньше, публиковать статьи по 4-5 страниц, хотя содержание многих статей можно было бы изложить на одной странице.
3. Раздел про гиперпараметры, датасеты и детали эксперимента может быть важен тем, кто будет имплементировать это на практике через месяц после выхода статьи, но через 5 лет станет не особенно интересен, хотя предложенный в статье метод может сохранить свою актуальность. Если же совсем не читать экспериментальный раздел, то можно упустить что-то важное.
4. При чтении статьи восприятие происходит однобоко. Непонятно, был ли предложенный в статье метод признан эффективным на практике, получил ли широкое распространение, или же вскоре был предложен более эффективный подход? Актуален ли метод по сей день, или если нет то почему? Часто не хватает взгляда со стороны от компетентного специалиста, желательно от нескольких.

Исходя из всего этого, систематизация и изложение статей в сокращенном варианте, как мне кажется, могло бы принести огромную пользу для сообщества. Конечно, многими прилагаются усилия в этом направлении, но этого пока недостаточно. Краткие изложения статей остаются бесплатной работой для энтузиастов, тогда как пользу они приносят вполне реальную. На мой взгляд, будь эта практика более широко распространена, и будь у людей больше интереса к исследованиям, мы могли бы уже создавать такие системы искусственного интеллекта, которую бы приносили огромную пользу человечеству (в медицине, физике, интернет-технологиях, оптимизации работы людей и, стало быть, сокращении рабочей недели) и которые пока что остаются лишь в области фантастики.